# 06_merge.ipynb

In [1]:
import pandas as pd
import geopandas as gpd
from dask.distributed import Client
import dask.dataframe as dd
from dask_jobqueue import SLURMCluster
import matplotlib.pyplot as plt
import contextily as cx

import dask_geopandas
INPUT_DATA_EVICTIONS = "../../data/02_intermediate/evictions.csv"
INPUT_DATA_TRACTS = "../../data/02_intermediate/tracts.csv"
INPUT_DATA_TAX_PARCELS = "../../data/02_intermediate/tax_parcels.gpkg"
INPUT_DATA_ZESTIMATES = "../../data/02_intermediate/zestimates.csv"
INPUT_DATA_CRIME = "../../data/01_raw/crime_incidents"
OUTPUT_DATA_UNRESTRICTED = "../../data/03_cleaned/unrestricted.csv"
OUTPUT_DATA_ZILLOW = "../../data/03_cleaned/zestimates_analysis.csv"
OUTPUT_DATA_CRIME = "../../data/03_cleaned/crime_analysis.csv"
VERBOSE = True
N_JOBS = 1

/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_173015/3932295706.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/late

## 1. Loading Evictions Data

In [2]:
# Load evictions data.
with open(INPUT_DATA_EVICTIONS, 'r') as file:
    all_column_names = set(file.readline().replace("\"", "").replace("\n", "").split(","))
to_drop = {'Accuracy Score', 'Accuracy Type', 'Number', 'Street', 'Unit Type', 'Unit Number',
           'State', 'Zip', 'Country', 'Source', 'Census Year', 'State FIPS', 'County FIPS',
           'Place Name', 'Place FIPS', 'Census Tract Code', 'Census Block Code', 'Census Block Group',
           'Metro/Micro Statistical Area Code', 'Metro/Micro Statistical Area Type',
           'Combined Statistical Area Code', 'Metropolitan Division Area Code', 'court_location',
           'defendant', 'defendant_atty', 'defendant_atty_address_apt',
           'defendant_atty_address_city', 'defendant_atty_address_name', 'defendant_atty_address_state',
           'defendant_atty_address_street', 'defendant_atty_address_zip', 'docket_history', 'execution', 'judgment_for',
           'judgment_total', 'latest_docket_date', 'plaintiff', 'plaintiff_atty', 'plaintiff_atty_address_apt',
           'plaintiff_atty_address_city', 'plaintiff_atty_address_name', 'plaintiff_atty_address_state',
           'plaintiff_atty_address_street', 'plaintiff_atty_address_zip', 'Metropolitan Division Area Name',
           'property_address_city', 'property_address_state', 'property_address_street',
           'property_address_zip'}
df = pd.read_csv(INPUT_DATA_EVICTIONS, usecols=set(all_column_names) - set(to_drop))
original_N = len(df)
if VERBOSE:
    print(f"Beginning with {original_N} observations.")

# Drop cases missing file_date.
mask = df['file_date'].notna()
if VERBOSE:
    print(
        f"Dropping {(~mask).sum()} observations where file_date is missing ({100 * (((~mask).sum()) / original_N):.3} percent "
        f"of original dataset).")
df = df.loc[mask, :]

# Add file month and year to dataset.
df.loc[:, 'file_month'] = pd.to_datetime(df['file_date']).dt.strftime('%Y-%m')
df.loc[:, 'file_year'] = pd.to_datetime(df['file_date']).dt.year

# Clean the values in the judgment_for_pdu variable.
judgment_for_pdu_replacement_dict = {"unknown": "Unknown",
                                     "plaintiff": "Plaintiff",
                                     "defendant": "Defendant"}
df.loc[:, "judgment_for_pdu"] = (df.loc[:, "judgment_for_pdu"]
                                           .replace(judgment_for_pdu_replacement_dict))

# Replace missing values in money judgment column with zeroes.
df.loc[:, 'judgment'] = df['judgment'].fillna(0)

# Rename duration to case_duration.
df = df.rename(columns={'duration': 'case_duration'})

# Drop malformed addresses.
if VERBOSE:
    print(f"Dropping {df['property_address_full'].str.contains('span, span span').sum()} observations which "
          f"have malformed addresses "
          f"({df['property_address_full'].str.contains('span, span span').sum() / original_N:.2f} "
          f"percent of observations).")
df = df.loc[~df['property_address_full'].str.contains("span, span span"), :]

# Drop addresses without latitude and longitude.
if VERBOSE:
    print(f"Dropping {df[['longitude', 'latitude']].isna().any(axis=1).sum()} evictions missing latitude "
          f"or longitude ({df[['longitude', 'latitude']].isna().any(axis=1).sum() / original_N:.2f}) "
          f"percent of observations.")
df = df.dropna(subset=['longitude', 'latitude'])

Beginning with 40759 observations.
Dropping 0 observations where file_date is missing (0.0 percent of original dataset).
Dropping 24 observations which have malformed addresses (0.00 percent of observations).
Dropping 1 evictions missing latitude or longitude (0.00) percent of observations.


## 2. Merging Evictions With Census Tract Characteristics

In [3]:
# Merge with census tract characteristics.
df = df.rename(columns={'Full FIPS (tract)': 'tract_geoid'})
df = df.merge(pd.read_csv(INPUT_DATA_TRACTS, dtype={'tract_geoid': float}),
                                  on='tract_geoid',
                                  how='left',
                                  validate='m:1')
if VERBOSE:
    print(f"Successfully merged {df['med_hhinc2016'].notna().sum()} observations "
          f"({df['med_hhinc2016'].notna().sum() / original_N:.2f} percent of observations) with census "
          f"tracts.")

Successfully merged 40732 observations (1.00 percent of observations) with census tracts.


## 3. Merging Evictions With Zestimates

In [4]:
df = pd.read_csv(INPUT_DATA_ZESTIMATES).merge(df,
                                                     on='case_number',
                                                     how='right',
                                                     validate='1:1')
if VERBOSE:
    successfully_matched_observations = (~df['2022-12'].isna()).sum()
    print(
        f"Successfully matched {successfully_matched_observations} evictions "
        f"({100 * (successfully_matched_observations / len(df)) :.2f} percent of observations) to "
        f"Zestimates.")

# Rename columns containing Zestimates.
years = [str(year) for year in range(2013, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = ["2012-12"] + [str(year) + "-" + str(month) for year in years for month in months]
for value_var in value_vars:
    df = df.rename(columns={value_var: value_var + "_zestimate"})
value_vars_zestimates = [value_var + "_zestimate" for value_var in value_vars]

Successfully matched 11496 evictions (28.22 percent of observations) to Zestimates.


## 4. Merging Evictions with Tax Parcels

In [5]:
# Request computing resources.
cluster = SLURMCluster(queue='batch',
                       cores=32,
                       memory='230 GB',
                       walltime='00:30:00',
                      scheduler_options={'dashboard_address': '8787'} )
cluster.scale(jobs=1)


In [6]:
client = Client(cluster)

In [7]:
df = gpd.GeoDataFrame(df,
                          geometry=gpd.points_from_xy(df['longitude'], df['latitude']))

df = df.set_crs("EPSG:4326", allow_override=True).to_crs('EPSG:26986')
ddf = dask_geopandas.from_geopandas(df, npartitions=N_JOBS).repartition(partition_size='50 MB')

tax_parcels_dgdf = dask_geopandas.read_file(INPUT_DATA_TAX_PARCELS, npartitions=N_JOBS, layer='layer').repartition(partition_size='50 MB')

ddf = dask_geopandas.sjoin(tax_parcels_dgdf, ddf, how='inner', predicate='contains')
ddf = ddf.loc[ddf['LOC_ID'] != "F_819960_2934955", :]  # Drop the eviction which erroneously merges to two parcels. 
ddf = ddf.drop(columns=['index_right']).compute()

# Append the evictions which could not be merged to tax parcels back into the Dask DataFrame.
evictions_without_parcels = df.loc[~(df['case_number'].isin(ddf['case_number'])), :]
ddf = pd.concat([ddf, evictions_without_parcels], axis=0)

## 5. Merge Evictions With Crime Data

In [8]:
ddf = dask_geopandas.from_geopandas(ddf, npartitions=N_JOBS)
ddf = ddf.repartition(partition_size='25 MB')  # Reduce partition size in preparation for spatial join.

crime_dgdf = (dd.read_csv(INPUT_DATA_CRIME + "/*.csv", dtype={'REPORTING_AREA': 'object', 'SHOOTING': 'object'})
                .dropna(subset=['Long', 'Lat', 'OCCURRED_ON_DATE'])
                .rename(columns={'OCCURRED_ON_DATE': 'month_of_crime_incident'})
                .drop(columns=['OFFENSE_CODE', 'OFFENSE_CODE_GROUP', 'OFFENSE_DESCRIPTION', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING', 'YEAR', 'MONTH',
                               'DAY_OF_WEEK', 'HOUR', 'UCR_PART', 'STREET', 'Location']))
crime_dgdf['month_of_crime_incident'] = dd.to_datetime(crime_dgdf['month_of_crime_incident'].str[:10]).dt.to_period("M").astype(str)
crime_dgdf = crime_dgdf.compute()

crime_dgdf = (gpd.GeoDataFrame(crime_dgdf, geometry=gpd.points_from_xy(crime_dgdf['Long'], crime_dgdf['Lat']))
                               .set_crs("EPSG:4326", allow_override=True)
                               .to_crs("EPSG:26986"))

crime_dgdf = dask_geopandas.from_geopandas(crime_dgdf, npartitions=N_JOBS).repartition(partition_size='25 MB')
ddf = dask_geopandas.sjoin(crime_dgdf,
                            ddf,
                            how='inner',
                            predicate='within').compute()

## 6. Aggregating Crime Data to Case-Month Level 

In [9]:
# Set aggregation function for each column.
# We will aggregate by the 'first' value for all columns except incident number, which we will count.
aggregate_by_first = ddf.columns.tolist()
aggregate_by_first.remove('INCIDENT_NUMBER')
aggregate_by_first.remove('case_number')
aggregate_by_first.remove('month_of_crime_incident')
agg_dict = {'INCIDENT_NUMBER': 'count'}
for column in aggregate_by_first:
    agg_dict[column] = 'first'
ddf = ddf.groupby(['case_number', 'month_of_crime_incident'])
ddf = ddf.aggregate(agg_dict)
ddf = ddf.reset_index()

In [10]:
# Pivot from wide to long.
ddf = ddf.rename(columns={'INCIDENT_NUMBER': 'crime_incidents'})
ddf = pd.pivot(ddf, index=['case_number'] + aggregate_by_first, columns=['month_of_crime_incident'], values='crime_incidents').reset_index()

/tmp/ipykernel_173015/2974544706.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ddf = pd.pivot(ddf, index=['case_number'] + aggregate_by_first, columns=['month_of_crime_incident'], values='crime_incidents').reset_index()
/tmp/ipykernel_173015/2974544706.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ddf = pd.pivot(ddf, index=['case_number'] + aggregate_by_first, columns=['month_of_crime_incident'], values='crime_incidents').reset_index()
/tmp/ipykernel_173015/2974544706.py:3: PerformanceWarning: DataFrame is highly fra

In [11]:
print(ddf.columns.tolist())

['case_number', 'Lat', 'Long', 'geometry', 'index_right', 'LOC_ID', '2012-12_zestimate', '2013-01_zestimate', '2013-02_zestimate', '2013-03_zestimate', '2013-04_zestimate', '2013-05_zestimate', '2013-06_zestimate', '2013-07_zestimate', '2013-08_zestimate', '2013-09_zestimate', '2013-10_zestimate', '2013-11_zestimate', '2013-12_zestimate', '2014-01_zestimate', '2014-02_zestimate', '2014-03_zestimate', '2014-04_zestimate', '2014-05_zestimate', '2014-06_zestimate', '2014-07_zestimate', '2014-08_zestimate', '2014-09_zestimate', '2014-10_zestimate', '2014-11_zestimate', '2014-12_zestimate', '2015-01_zestimate', '2015-02_zestimate', '2015-03_zestimate', '2015-04_zestimate', '2015-05_zestimate', '2015-06_zestimate', '2015-07_zestimate', '2015-08_zestimate', '2015-09_zestimate', '2015-10_zestimate', '2015-11_zestimate', '2015-12_zestimate', '2016-01_zestimate', '2016-02_zestimate', '2016-03_zestimate', '2016-04_zestimate', '2016-05_zestimate', '2016-06_zestimate', '2016-07_zestimate', '2016-08

In [12]:
# Append the evictions which could not be merged to tax parcels back into the DataFrame.
evictions_without_parcels = df.loc[~(df['case_number'].isin(ddf['case_number'])), :]
ddf = pd.concat([ddf, evictions_without_parcels], axis=0)

/gpfs/home/ashanmu1/seniorthesis/venv/lib/python3.10/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as NAD83 / Massachusetts Mainland (the single non-null crs provided).
  warnings.warn(


In [13]:
# Rename columns containing crime incident counts.
years = [str(year) for year in range(2015, 2023)]
months = ["0" + str(month) for month in range(1, 10)] + [str(month) for month in range(10, 13)]
value_vars = [str(year) + "-" + str(month) for year in years for month in months]
for value_var in value_vars:
    ddf = ddf.rename(columns={value_var: value_var + "_crimes"})
value_vars_crime = [value_var + "_crimes" for value_var in value_vars]

In [14]:
# Replace missing crime data with zero for evictions that were not matched to crimes.
ddf.loc[:, value_vars_crime] = ddf[value_vars_crime].fillna(0)

['case_number', 'Lat', 'Long', 'geometry', 'index_right', 'LOC_ID', '2012-12_zestimate', '2013-01_zestimate', '2013-02_zestimate', '2013-03_zestimate', '2013-04_zestimate', '2013-05_zestimate', '2013-06_zestimate', '2013-07_zestimate', '2013-08_zestimate', '2013-09_zestimate', '2013-10_zestimate', '2013-11_zestimate', '2013-12_zestimate', '2014-01_zestimate', '2014-02_zestimate', '2014-03_zestimate', '2014-04_zestimate', '2014-05_zestimate', '2014-06_zestimate', '2014-07_zestimate', '2014-08_zestimate', '2014-09_zestimate', '2014-10_zestimate', '2014-11_zestimate', '2014-12_zestimate', '2015-01_zestimate', '2015-02_zestimate', '2015-03_zestimate', '2015-04_zestimate', '2015-05_zestimate', '2015-06_zestimate', '2015-07_zestimate', '2015-08_zestimate', '2015-09_zestimate', '2015-10_zestimate', '2015-11_zestimate', '2015-12_zestimate', '2016-01_zestimate', '2016-02_zestimate', '2016-03_zestimate', '2016-04_zestimate', '2016-05_zestimate', '2016-06_zestimate', '2016-07_zestimate', '2016-08

KeyError: "['2015-01_crimes', '2015-02_crimes', '2015-03_crimes', '2015-04_crimes', '2015-05_crimes'] not in index"

In [ ]:
# TODO: Save unrestricted dataset.
ddf = pd.DataFrame(ddf.drop(columns='geometry'))

In [ ]:
# TODO: Save Zillow sample.

In [ ]:
# TOOD: Save crime sample.
